In [1]:
from cobra.io import read_sbml_model
from pathlib import Path
import os
import cobra

cobra.Configuration().solver = "gurobi"

# change working directory to context_specific_models folder
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

# create a dictionary with the filenames (keys) and models (values)
folder = Path().cwd()
files = [f for f in os.listdir(folder / 'cs_mods') if '.xml'in f]
models = {f:read_sbml_model(folder / 'cs_mods' / f ) for f in files}
k = list(models.keys())

# add model names as IDs
models_names = ['Human1', 'Schwan (1)', 'Schwan (2)', 'Melanocytes', 'Endothelial','Muscle','Fibroblasts',\
                'RPE','B cells','T + NK cells', 'Monocytes + Macrophages','Mast','Retina','RPE + Choroid',\
                'PR rod','Amacrine','Bipolar','PR cone', 'Horizontal', 'Müller','Ganglion']

for i in range(len(models)):
    models[k[i]].id = models_names[i]

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-03


In [2]:
import pandas as pd

mod = models[k[0]].copy()

# add back any lost exchange reactions
def add_all_EX_rxns(basis_model,model):
    EX_rxns = [r for r in basis_model.reactions if len(r.products)==0]
    EX_rxns_model = [r.id for r in model.reactions if len(r.products)==0]
    missing_rxns = [r.copy() for r in EX_rxns if r.id not in EX_rxns_model]
    model.add_reactions(missing_rxns)
    return model

# import excel file with info on exchange upper bounds 
# file contains 'id' col with IDs and an 'upper_bound' col with upper bounds to modify
bounds = pd.read_excel(folder / 'rxn_bounds' / 'R3D301_EX_rxns.xlsx')
bounds = bounds[bounds['upper_bound']>0] # select entries with non-zero upper bounds

# add ATP demand reactions, lost exchange reactions to all models
for k_i in k:
    models[k_i].add_reactions([mod.reactions.get_by_id('MAR03964').copy()])
    models[k_i] = add_all_EX_rxns(mod,models[k_i])
    for r in [r for r in models[k_i].reactions if len(r.products)==0]: r.bounds = (0,1000)     # close lower bound for all exchange reactions (metabolites can leave the system but not enter)
    for i in range(len(bounds)): models[k_i].reactions.get_by_id(bounds['id'][i]).bounds = (-bounds['upper_bound'][i],1000)     # set bounds Exchange reactions (lower bounds -1000, upper bounds based on file info)

Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpaefd2dmm.lp
Reading time = 0.11 seconds
: 8369 rows, 26140 columns, 111668 nonzeros


Ignoring reaction 'MAR03964' since it already exists.


In [3]:
mod.reactions.get_by_id('MAR03964')

Reaction identifier,MAR03964
Name,ATP phosphohydrolase
Memory address,0x19abac0e560
Stoichiometry,MAM01371c + MAM02040c --> MAM01285c + MAM02039c + MAM02751c ATP + H2O --> ADP + H+ + Pi
GPR,
Lower bound,0.0
Upper bound,1000.0


In [4]:
# reactions containing atp[c]
atp_rxns = list(set([r for r in list(mod.metabolites.get_by_id('MAM01371c').reactions)]))
df_atp_rxns = pd.DataFrame([[r.id, r.name, \
              r.build_reaction_string(use_metabolite_names = True)]\
              for r in atp_rxns])
df_atp_rxns.to_clipboard(excel=True, sep=None)

In [5]:
solution = []
for k_i in k:
    models[k_i].objective = models[k_i].reactions.get_by_id('MAR03964').flux_expression
    s = models[k_i].optimize()
    solution.append(s.objective_value) 
pd.DataFrame(k,solution)

,0
96.843248,Human-GEM.xml
80.000000,model_00_MeanExpression_SchwannCells_1__VoigtE...
80.000000,model_01_MeanExpression_SchwannCells_2__VoigtE...
0.000000,model_02_MeanExpression_Melanocytes__VoigtEtAl...
0.000000,model_03_MeanExpression_EndothelialCells__Voig...
80.000000,model_04_MeanExpression_SmoothMuscle__VoigtEtA...
1.116667,model_05_MeanExpression_Fibroblasts__VoigtEtAl...
81.889487,model_06_MeanExpression_RPE__VoigtEtAl2019_.xml
3.637094,model_07_MeanExpression_BCells__VoigtEtAl2019_...
0.000000,model_08_MeanExpression_TCells_NKCells__VoigtE...


In [6]:
# open Exchange reactions
for k_i in k:
    for r in [r for r in models[k_i].reactions if len(r.products)==0]: r.bounds = (-.0001,1000)
solution = []
for k_i in k:
    models[k_i].objective = models[k_i].reactions.get_by_id('MAR03964').flux_expression
    s = models[k_i].optimize()
    solution.append(s.objective_value) 
pd.DataFrame(k,solution)

,0
0.135502,Human-GEM.xml
0.010015,model_00_MeanExpression_SchwannCells_1__VoigtE...
0.009100,model_01_MeanExpression_SchwannCells_2__VoigtE...
0.006542,model_02_MeanExpression_Melanocytes__VoigtEtAl...
0.005958,model_03_MeanExpression_EndothelialCells__Voig...
0.008559,model_04_MeanExpression_SmoothMuscle__VoigtEtA...
0.006981,model_05_MeanExpression_Fibroblasts__VoigtEtAl...
0.011129,model_06_MeanExpression_RPE__VoigtEtAl2019_.xml
0.007561,model_07_MeanExpression_BCells__VoigtEtAl2019_...
0.005707,model_08_MeanExpression_TCells_NKCells__VoigtE...


In [7]:
# open Exchange reactions
for k_i in k:
    for r in [r for r in models[k_i].reactions if len(r.products)==0]: r.bounds = (-.1,1000)
solution = []
for k_i in k:
    models[k_i].objective = models[k_i].reactions.get_by_id('MAR03964').flux_expression
    s = models[k_i].optimize()
    solution.append(s.objective_value) 
pd.DataFrame(k,solution)

,0
135.501677,Human-GEM.xml
10.014763,model_00_MeanExpression_SchwannCells_1__VoigtE...
9.100179,model_01_MeanExpression_SchwannCells_2__VoigtE...
6.541677,model_02_MeanExpression_Melanocytes__VoigtEtAl...
5.957493,model_03_MeanExpression_EndothelialCells__Voig...
8.558923,model_04_MeanExpression_SmoothMuscle__VoigtEtA...
6.980705,model_05_MeanExpression_Fibroblasts__VoigtEtAl...
11.129167,model_06_MeanExpression_RPE__VoigtEtAl2019_.xml
7.561048,model_07_MeanExpression_BCells__VoigtEtAl2019_...
5.707142,model_08_MeanExpression_TCells_NKCells__VoigtE...


In [8]:
mod.metabolites.get_by_id('MAM01371c')

Metabolite identifier,MAM01371c
Name,ATP
Memory address,0x19ab9bd9db0
Formula,C10H12N5O13P3
Compartment,c
In 898 reaction(s),"MAR04476, MAR12164, MAR12330, MAR12545, MAR04480, MAR12394, MAR01451, MAR12398, MAR04133, MAR00395, MAR12013, MAR10073, MAR00335, MAR12750, MAR12821, MAR01314, MAR12339, MAR04350, MAR12547,..."


In [9]:
import pandas as pd

l = [list(r.annotation.keys()) for r in models[k[0]].reactions]
annotation_keys = list(set([item for sublist in l for item in sublist]))

def get_annotation_ids(x, annotation_keys):
    annotation = []
    for k in annotation_keys:
        if k in x.annotation.keys():
            annotation.append(x.annotation[k])
        else:
            annotation.append('')
    return annotation

In [10]:
# get met IDs

l = [list(m.annotation.keys()) for m in  models[k[0]].metabolites]
annotation_keys = list(set([item for sublist in l for item in sublist]))

mets = pd.DataFrame([[m.id]+get_annotation_ids(m,annotation_keys)+\
                     [m.name,m.formula,m.charge,m.compartment,[r.id for r in m.reactions]]\
                      for m in models[k[0]].metabolites],columns=['Human1.metabolite']+annotation_keys+\
                    ['name','formula','charge','compartment','reactions'])

mets.to_clipboard(excel=True, sep=None)
mets

,Human1.metabolite,kegg.compound,bigg.metabolite,metanetx.chemical,lipidmaps,inchi,chebi,vmhmetabolite,sbo,hmdb,pubchem.compound,name,formula,charge,compartment,reactions
0,MAM00001c,C00964,carveol,MNXM45735,,,CHEBI:15389,carveol,SBO:0000247,,,(-)-trans-carveol,C10H16O,0,c,"[MAR08597, MAR08596]"
1,MAM00001e,C00964,carveol,MNXM45735,,,CHEBI:15389,carveol,SBO:0000247,,,(-)-trans-carveol,C10H16O,0,e,"[MAR09277, MAR08597]"
2,MAM00002c,C09880,appnn,MNXM163755,,,CHEBI:36740,appnn,SBO:0000247,HMDB06525,6654,(+)-alpha-pinene,C10H16,0,c,"[MAR08601, MAR08600]"
3,MAM00002e,C09880,appnn,MNXM163755,,,CHEBI:36740,appnn,SBO:0000247,HMDB06525,6654,(+)-alpha-pinene,C10H16,0,e,"[MAR09267, MAR08600]"
4,MAM00003c,,,"[MNXM150165, MNXM27815]",LMFA01030283,,CHEBI:78990,M00003,SBO:0000247,,,(10Z)-heptadecenoic acid,C17H31O2,-1,c,"[MAR00685, MAR00241, MAR00243, MAR09738, MAR10..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8364,MAM10039c,,,,,,,,SBO:0000247,,,dehydrocholoyl-CoA,C45H64N7O20P3S,-4,c,"[MAR10111, MAR10113, MAR10112, MAR10114]"
8365,MAM10040c,,,,,,,,SBO:0000247,,,hyodeoxycholoyl-CoA,C45H70N7O19P3S,-4,c,"[MAR10122, MAR10123, MAR10121, MAR10124]"
8366,MAM10041c,,,,,,,,SBO:0000247,,119046,tauro-hyodeoxycholic acid,C26H44NO6S,-1,c,"[MAR10127, MAR10123, MAR10125]"
8367,MAM10041e,,,,,,,,SBO:0000247,,119046,tauro-hyodeoxycholic acid,C26H44NO6S,-1,e,"[MAR10127, MAR10132, MAR10125]"


In [11]:
# get rxn IDs

rxns = pd.DataFrame([[r.id]+get_annotation_ids(r,annotation_keys)+\
                     [r.name,r.subsystem,r.compartments,r.reaction,r.build_reaction_string(use_metabolite_names = True),r.gpr]\
                      for r in models[k[0]].reactions],\
                    columns=['Human1.reaction']+annotation_keys+['name','subsystem','compartments','reaction (IDs)','reaction (names)','GPR'])

rxns.to_clipboard(excel=True, sep=None)
rxns

,Human1.reaction,kegg.compound,bigg.metabolite,metanetx.chemical,lipidmaps,inchi,chebi,vmhmetabolite,sbo,hmdb,pubchem.compound,name,subsystem,compartments,reaction (IDs),reaction (names),GPR
0,MAR03905,,,,,,,,SBO:0000176,,,ethanol:NAD+ oxidoreductase,Glycolysis / Gluconeogenesis,{c},MAM01796c + MAM02552c --> MAM01249c + MAM02039...,ethanol + NAD+ --> acetaldehyde + H+ + NADH,ENSG00000147576 or ENSG00000172955 or ENSG0000...
1,MAR03907,,,,,,,,SBO:0000176,,,Ethanol:NADP+ oxidoreductase,Glycolysis / Gluconeogenesis,{c},MAM01796c + MAM02554c --> MAM01249c + MAM02039...,ethanol + NADP+ --> acetaldehyde + H+ + NADPH,ENSG00000117448
2,MAR04097,,,,,,,,SBO:0000176,,,Acetate:CoA ligase (AMP-forming),Glycolysis / Gluconeogenesis,{c},MAM01252c + MAM01371c + MAM01597c --> MAM01261...,acetate + ATP + CoA --> acetyl-CoA + AMP + PPi,ENSG00000131069
3,MAR04099,,,,,,,,SBO:0000176,,,Acetate:CoA ligase (AMP-forming),Glycolysis / Gluconeogenesis,{m},MAM01252m + MAM01371m + MAM01597m --> MAM01261...,acetate + ATP + CoA --> acetyl-CoA + AMP + PPi,ENSG00000111058 or ENSG00000154930
4,MAR04108,,,,,,,,SBO:0000176,,,acetyl adenylate:CoA acetyltransferase,Glycolysis / Gluconeogenesis,{c},MAM01257c + MAM01597c --> MAM01261c + MAM01334...,acetyl adenylate + CoA --> acetyl-CoA + AMP + H+,ENSG00000131069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13065,MAR10132,,,,,,,,SBO:0000627,,,,Exchange/demand reactions,{e},MAM10041e <=>,tauro-hyodeoxycholic acid <=>,
13066,MAR13083,,,,,,,,SBO:0000176,,,,Metabolism of other amino acids,{c},MAM02039c + MAM02157c + MAM02553c + MAM02630c ...,H+ + hypotaurine + NADH + O2 --> H2O + NAD+ + ...,ENSG00000010932
13067,MAR13084,,,,,,,,SBO:0000185,,,,Transport reactions,"{c, n}",MAM01639c <=> MAM01639n,dAMP <=> dAMP,
13068,MAR13085,,,,,,,,SBO:0000185,,,,Transport reactions,"{c, n}",MAM01686c <=> MAM01686n,dGMP <=> dGMP,
